## Parse Yosys JSON netlist

In [341]:
import json

In [342]:
# generate pinouts
import glob
import os
paths = glob.glob('../74xx/**/**.dig')
paths

import xml.etree.ElementTree as ET

pinouts = {}
sizes = {}
for path in paths:
    chip_name = os.path.basename(path)
    chip_name = chip_name[:chip_name.find('.dig')]
    p = ET.parse(path).getroot()
    pinout = {}
    size = 0
    visualElements = None # will contain all pins
    for child in p:
        if child.tag == 'visualElements':
            visualElements = child
            break

    for ve_list in visualElements:
        valid = False
        pin_name = None
        pin_num = None
        for ve_list_item in ve_list:
            if ve_list_item.tag == 'elementName':
                t = ve_list_item.text
                if t == 'In' or t == 'Out':
                    valid = True
            if ve_list_item.tag == 'elementAttributes':
                for a in ve_list_item:
                    check = False
                    label = None
                    for b in a:
                        if b.tag == 'string':
                            if not b.text == 'Label':
                                label = b.text
                            else:
                                check = True
                    if check:
                        pin_name = label

                for a in ve_list_item:
                    check1 = False
                    num = None
                    for b in a:
                        if b.tag == 'string':
                            if not b.text == 'pinNumber':
                                num = b.text
                            else:
                                check1 = True
                    if check1:
                        pinout[pin_name] = num
                        if int(num) > size:
                            size = int(num)
    sizes[chip_name] = size
    pinouts[chip_name] = pinout
    # print(chip_name, size, pinout, sep='    \t')

In [343]:
footprints = {
    16: 'DIP-16_W7.62mm',
    14: 'DIP-14_W7.62mm'
    }

In [344]:
with open('../out.json') as fp:
    js = json.load(fp)

modules = js['modules']

top = modules['dumb_op_card']

# The card IO wires
print('Ports:', top['ports'])
print()

netlist = {}

for k in top['ports']:
    port = top['ports'][k]
    bits = port['bits']
    # print(k, bits)
    for i in range(len(bits)):
        bit = bits[i]
        name = k
        if i > 0:
            name = name + str(i)
        netlist[bit] = name

# Pass 1 - add all nets
for c in top['cells'].keys():
    type = top['cells'][c]['type']
    if not type.startswith('\\74') or type.startswith('\\74AC'):
        # print('Error: ' + type + ' is not a 74-series IC! Skipping logic implementation\n')
        continue

    type = type.replace('\\', '') # remove backslashes

    wires = top['cells'][c]['connections']

    for k in wires.keys():
        wire = wires[k]
        assert len(wire) == 1
        wire = int(wire[0])
        # print(k, wire)
        if wire not in netlist.keys():
            netlist[wire] = 'n' + str(wire)

# TODO Here: add kicad nets

# Pass 2 - add chips
i = 0
for c in top['cells'].keys():
    type = top['cells'][c]['type']
    if not type.startswith('\\74') or type.startswith('\\74AC'):
        # print('Error: ' + type + ' is not a 74-series IC! Skipping logic implementation\n')
        continue

    type = type.replace('\\', '') # remove backslashes
    
    size = sizes[type]            # get pinout size
    wires = top['cells'][c]['connections']
    footprint = footprints[size]

    # TODO here: create footprint and position

    for k in wires.keys():
        wire = wires[k]
        assert len(wire) == 1
        wire = int(wire[0])
        # print(k, wire)
        # TODO here: add nets

    i += 1
    # print(type, end=':\n')
    # print('\t', wires)
    # print('\t', pinouts[type])


Ports: {'clk': {'direction': 'input', 'bits': [2]}, '~rst': {'direction': 'input', 'bits': [3]}, '~int': {'direction': 'input', 'bits': [4]}, 'VCC_5V': {'direction': 'input', 'bits': [5]}, 'GND': {'direction': 'input', 'bits': [6]}, 'VCC_3V3': {'direction': 'input', 'bits': [7]}, '~we': {'direction': 'input', 'bits': [8]}, '~oe': {'direction': 'input', 'bits': [9]}, 'addr': {'direction': 'input', 'bits': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}, 'aux-bus': {'direction': 'input', 'bits': [26, 27, 28, 29, 30, 31, 32, 33]}, 'data': {'direction': 'output', 'bits': [34, 35, 36, 37, 34, 35, 36, 37]}}

